In [1]:
import fg_make_file as fg
import cmb_make_file as cmb
import Delta_map as dm
import numpy as np
import healpy as hp
import warnings
warnings.filterwarnings("ignore")

Using CAMB 1.5.2 installed at /Users/ikumakiyoshi/.julia/conda/3/lib/python3.10/site-packages/camb


In [14]:
random_seed_cmb = 5123
seed_syn = 12345
nside = 2**2

cmb_r_0 = cmb.cmb_make_file(nside, 0, random_seed_cmb, seed_syn)

"""
r_elements = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.]
r_elements_name = ["1", "01", "001", "0001", "00001", "000001", "0000001", "0"]

# rturn CMB (r = i)
# name : cmb_r_i
for j, i in enumerate(r_elements):
    variable_name = f'cmb_r_{r_elements_name[j].replace(".", "_")}'
    locals()[variable_name] = cmb.cmb_make_file(nside, i, random_seed_cmb, seed_syn)
    """

'\nr_elements = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.]\nr_elements_name = ["1", "01", "001", "0001", "00001", "000001", "0000001", "0"]\n\n# rturn CMB (r = i)\n# name : cmb_r_i\nfor j, i in enumerate(r_elements):\n    variable_name = f\'cmb_r_{r_elements_name[j].replace(".", "_")}\'\n    locals()[variable_name] = cmb.cmb_make_file(nside, i, random_seed_cmb, seed_syn)\n    '

In [15]:
f_elements = [40, 50, 100, 140, 280, 402]

#f_elements = [40, 100, 140]

for k in f_elements:
    synch = f'synch_{str(k).replace(".", "_")}'
    locals()[synch] = fg.fg_make_file(k, nside)[0]

    dust = f'dust_{str(k).replace(".", "_")}'
    locals()[dust] = fg.fg_make_file(k, nside)[1]

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [10]:
#f_elements = [40, 50, 100, 140, 280, 402]
#f_elements = [40, 100, 140]
#nside = 2**4

#D_1 =  dm.calc_D_matrix(f_elements, "d1 and s1", nside)
#D_1 =  dm.calc_D_matrix(f_elements, "s1", nside)

In [11]:
#D_1

In [12]:
#dm.D_element(100)

In [17]:
# which_model = "s1", "d1", or "s1 and d1"

def make_data_m(cmb, freq_band, which_model, nside):

    band = int(len(freq_band))

    N_pix = hp.nside2npix(nside)

    data_m = np.zeros(int(2*N_pix*band))

    if which_model == "s1":
        
        for ii, freq in enumerate(freq_band):

            # data m[I, Q, U]
            m = fg.fg_make_file(freq, nside)[0] + cmb
            
            data_m[int((2*ii)*N_pix) : int((2*ii+1)*N_pix)] = m[1]
            data_m[int((2*ii+1)*N_pix) : int(((2*ii+2))*N_pix)] = m[2]
            
    elif which_model == "d1":

        for ii, freq in enumerate(freq_band):

            # data m[I, Q, U]
            m = fg.fg_make_file(freq, nside)[1] + cmb

            data_m[int((2*ii)*N_pix) : int((2*ii+1)*N_pix)] = m[1]
            data_m[int((2*ii+1)*N_pix) : int(((2*ii+2))*N_pix)] = m[2]     

    elif which_model == "d1 and s1":

        for ii, freq in enumerate(freq_band):

            # data m[I, Q, U]
            m = fg.fg_make_file(freq, nside)[0] + fg.fg_make_file(freq, nside)[1] + cmb

            data_m[int((2*ii)*N_pix) : int((2*ii+1)*N_pix)] = m[1]
            data_m[int((2*ii+1)*N_pix) : int(((2*ii+2))*N_pix)] = m[2]

    else:
        print("正しいモデル入れてね")

    return data_m

In [18]:
# which_model = "s1", "d1", or "s1 and d1"
f_elements = [40, 50, 100, 140, 280, 402]

#return m_[Q, U]
data_m = make_data_m(cmb_r_0, f_elements, "d1 and s1", nside)

In [20]:
data_m.shape

(2304,)

In [47]:
import matplotlib.pyplot as plt
import healpy as hp
%matplotlib inline

#hp.mollview(cmb_r_001[2], title=" map", unit = "μK")
#hp.mollview(dust_40[0], title=" map", unit = "μK")

In [ ]:
D = np.block([[diag, d_40 * diag, dd_40 * diag], [diag, d_100 * diag, dd_100 * diag], [diag, d_140 * diag, dd_140 * diag]])

In [80]:
import numpy as np

A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [3, 4]])
C = np.array([[7, 9], [3, 4]])
D = np.array([[11, 12], [13, 14]])


# numpy.blockを使って結合
result = np.block([[A, C],[B, D]])

print(result)

[[ 1  2  7  9]
 [ 3  4  3  4]
 [ 5  6 11 12]
 [ 3  4 13 14]]


In [85]:
DD = []
DD.append([A,C])
DD.append([B,D])

np.block(DD)

array([[ 1,  2,  7,  9],
       [ 3,  4,  3,  4],
       [ 5,  6, 11, 12],
       [ 3,  4, 13, 14]])

In [6]:
def D_element(freq, freq_b = 23 * 10**9, beta = -3.):

    # x = (f / T) * (h / k)
    x = (freq / 2.725) / (2.083661912 * 10**10)
    
    g_freq = ((np.exp(x) - 1)**2) / (np.exp(x) * x**2) * 1000.0

    
    d = g_freq * (freq/freq_b)**(beta)

    dd = g_freq * (freq/freq_b)**(beta) * np.log(freq/freq_b)
    
    return d, dd


def  calc_D_matrix(freq_band, which_model, nside):

    N_pix = hp.nside2npix(nside)

    diag = np.identity(2 * N_pix)

    D_blocks = []

    for freq in freq_band:

        d, dd = D_element(freq * 10**9)

        D_blocks.append([diag, d * diag, dd * diag])

    D_matrix = np.block(D_blocks)

    return D_matrix
        

D_1 =  dm.calc_D_matrix(f_elements, "d1 and s1", nside)

# freq_b = 23 GHz, beta = -3.0
d_40, dd_40 = D_element(40 * 10**9)
d_100, dd_100 = D_element(100 * 10**9)
d_140, dd_140 = D_element(140 * 10**9)


# D Matrix
N_pix = hp.nside2npix(nside)
diag = np.identity(2 * N_pix)

# Block Matrix, CMB + FG
D = np.block([[diag, d_40 * diag, dd_40 * diag], [diag, d_100 * diag, dd_100 * diag], [diag, d_140 * diag, dd_140 * diag]])

print(D.shape)

print(D_1.shape)

(18432, 18432)
(36864, 36864)


In [22]:
import Delta_map as dm

D_2 = dm.calc_D_matrix(f_elements, "s1", nside)

In [99]:
D_1-D_2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [2]:
f_elements = [40, 50, 100, 140, 280, 402]
nside = 2**2

D_1 =  dm.calc_D_matrix(f_elements, "d1 and s1", nside)

In [4]:
D_1.shape

(2304, 2304)

In [5]:
D_1

array([[  1.       ,   0.       ,   0.       , ...,  -0.       ,
         -0.       ,  -0.       ],
       [  0.       ,   1.       ,   0.       , ...,  -0.       ,
         -0.       ,  -0.       ],
       [  0.       ,   0.       ,   1.       , ...,  -0.       ,
         -0.       ,  -0.       ],
       ...,
       [  0.       ,   0.       ,   0.       , ..., 101.1321824,
          0.       ,   0.       ],
       [  0.       ,   0.       ,   0.       , ...,   0.       ,
        101.1321824,   0.       ],
       [  0.       ,   0.       ,   0.       , ...,   0.       ,
          0.       , 101.1321824]])